## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# get modules
from air_quality import *
from fisheries import *
from noise import *
from recreation import *
from scenic_resources import *
from soil_conservation import *
from transportation_sustainablecommunities import *
from vegetation_preservation import *
from water_quality import *
from wildlife import *

## Soil Conservation

In [5]:
# get land capability data
dfImpChg, dfImp2019 = get_soil_conservation_data_web()

In [ ]:
df

In [6]:
# transform new coverage data
dfImpNew = transform_new_coverage_data()
# transform soil conservation data
dfImpOld = transform_soil_conservation_data(dfImpChg, dfImp2019)
df = add_new_coverage(dfImpOld, dfImpNew)
df

,Land Capability,Acres of Hard Surface 2019,Acres of Soft Surface 2019,Acres of Coverage 2019,Percent Hard 2019,Percent Soft 2019,Percent Impervious 2019,Total Acres,Threshold Value,Threshold Acres,SqFt,Acres of New Coverage,Acres of Coverage 2023
0,1A,0.055125,0.146009,0.201134,0.051484,1.363665e-01,0.187851,107.070923,1%,1.070709,-2744.000000,-0.062994,0.138140
1,1B,0.415023,0.585477,1.000500,0.084274,1.188857e-01,0.203159,492.470733,1%,4.924707,-69744.000000,-1.601102,-0.600602
2,1C,0.086758,0.711088,0.797845,0.003319,2.720716e-02,0.030527,2613.606127,1%,26.136061,0.000000,0.000000,0.797845
3,2,0.007584,0.015684,0.023268,0.001786,3.693077e-03,0.005479,424.688625,1%,4.246886,0.000000,0.000000,0.023268
4,3,0.537689,1.552243,2.089932,0.067422,1.946398e-01,0.262062,797.495518,5%,39.874776,0.000000,0.000000,2.089932
5,4,0.064962,0.111007,0.175968,0.026448,4.519520e-02,0.071644,245.616502,20%,49.123300,54280.806453,1.246116,1.422084
6,5,0.382236,0.061570,0.443807,0.153653,2.475037e-02,0.178404,248.765202,25%,62.191300,-24803.930107,-0.569420,-0.125613
7,6,0.084123,0.000136,0.084260,0.033513,5.433126e-05,0.033568,251.015732,30%,75.304720,308022.365594,7.071221,7.155480
8,7,0.657318,0.010311,0.667629,0.618231,9.697659e-03,0.627929,106.322358,30%,31.896707,915907.564514,21.026344,21.693973
9,NaN,1.388481,0.000472,1.388953,0.001131,3.841831e-07,0.001131,122789.102720,0%,0.000000,NaN,NaN,NaN


In [7]:
df.sort_values(by="Land Capability")

colors = ['#D1FF73','#FFFF00','#4CE600','#4C7300', 
          '#0084A8', '#FFD37F','#FFAA00','#CD8966', '#734C00']

fig = px.bar(df, y="Land Capability", x="Total Acres", color="Land Capability", 
             orientation="h", hover_name="Land Capability",
             color_discrete_sequence=colors ,
             title="Land Capability"
            )

fig.update_traces(hovertemplate='%{y}<br>%{x:,.0f} acres<extra></extra>')

fig.update_layout(font_family=font, template=template, showlegend=True)
fig.show()

In [10]:

colors = ['lightslategray',] * 9
colors[0] = '#279bdc'

# create threshold lines
fig = go.Figure(go.Scatter(
    y=df['Threshold Acres'],
    x=df['Land Capability'],
    name= "Threshold",
    line=dict(color='#333333', width=3),
    mode='markers',
    marker_symbol='line-ew',
    marker_line_color="midnightblue", 
    marker_color="lightskyblue", 
    marker_line_width=2, 
    marker_size = 36,
    customdata=df['Threshold Value'],
    hovertemplate='Threshold:<br>%{customdata} coverage allowed<br>or %{y:,.0f} acres<extra></extra>'
))

# create coverage bars
fig.add_trace(go.Bar(
    y=df['Acres of Coverage 2019'],
    x=df['Land Capability'],
    name= "Coverage in Class 1A",
    marker_color=colors,
#     marker_color='rgb(188,202,200)', 
    marker_line_color='rgb(88,48,10)',
    opacity=0.6,
    hovertemplate='<b>%{y:,.0f} acres</b> covered<extra></extra>'
))

# set layout
fig.update_layout(title='Impervious Cover in Land Capability Class 1A',
                    font_family=font,
                    template=template,
                    legend_title_text='',
                    showlegend=True,
                    hovermode="x unified",
                    barmode='overlay',
                    xaxis = dict(
                        tickmode = 'linear',
                        title_text='Land Capability Class',
#                         range=[-1,4.5]
                    ),
                    yaxis = dict(
                        title_text='Acres',
                        tickmode = 'linear',
                        rangemode="tozero",
                        range= [0,100],
                        tick0 = 0,
                        dtick = 10,
                        tickformat=",.0"
                    )
                 )

fig.show()
# save to HTML
fig.write_html(os.path.join(workspace, "SoilConservation_LandCapability_Coverage_1A.html"))

NameError: name 'workspace' is not defined

In [ ]:
# plot 1A - soil conservation
plot_soil_conservation(df, landcap = '1A')

## Water Quality

In [ ]:
# secchi depth data and chart
df = get_secchi_data_sql()
# df = get_secchi_data_web()
plot_secchi_depth(df, draft=False)

In [ ]:
# watercraft inspections
df = get_watercraft_inspection_data_web()
plot_watercraft_inspections(df, draft=False)

## Wildlife

In [ ]:
# goshawk data and chart
df = get_wildlife_data_web()
plot_goshawk_data(df, draft=False)

In [ ]:
# osprey data and chart
df = get_wildlife_data_web()
plot_osprey_data(df, draft=True)

In [ ]:
# bald eagle winter roost data and chart
df = get_wildlife_data_web()
plot_bald_eagle_winter(df, draft=False)

In [ ]:
# peregine falcon data and chart
df = get_wildlife_data_web()
plot_falcon_data(df, draft=False)

In [ ]:
# bald eagle summer roost data and chart
df = get_wildlife_data_web()
plot_bald_eagle_summer(df, draft=False)

In [ ]:
#waterfowl data and chart
df = get_waterfowl_data_web()
plot_waterfowl_data(df, draft=False)